In [1]:
# Add utils libraries to program 
import sys
sys.path.append('/home/class_NI2021/ctxctl_contrib_2023')

# Imports all necessary libraries (samna and utils)
import samna
import samna.dynapse1 as dyn1
from dynapse1constants import *
import dynapse1utils as ut
import netgen as n
import params
import time
import math
import numpy as np
from datetime import datetime

# Imports libraries for STDP
from params import set_stdp_params
from stdp import Stdp
import stdp_utils 

In [ ]:
# Check which dynapse are free
devices = samna.device.get_unopened_devices()
for i in range(len(devices)):
        print("["+str(i)+"]: ", devices[i], "serial_number", devices[i].serial_number)

In [ ]:
# Open DYNAP-SE1 board selected (by ports) and get Dynapse1Model, without GUI (GUI = False)
model, _ = ut.open_dynapse1(gui=False, sender_port=17345, receiver_port=15334, select_device=True) # dynapse = 11

In [2884]:
# Needed functions
def get_bin_representation(input_ids):
    binary_ids = []
    max_num_digits = 0
    # Create a copy of input neurons ids to reverse and begin with the bigger one
    neurons_id = input_ids.copy()
    neurons_id.reverse()
    # For each neuron id
    for id in neurons_id:
        res = id
        binary_id = []
        # Calculate the binary representation
        while res > 1:
            binary_id.append(res % 2)
            res = res // 2
        if res == 0:
            binary_id.append(0)
        else:
            binary_id.append(1)
        # Adjust all binary representations to the number with the max number of digits
        if max_num_digits == 0:
            max_num_digits = len(binary_id)
        elif max_num_digits > len(binary_id):
            diff = max_num_digits - len(binary_id)
            for i in range(diff):
                binary_id.append(0)
        # Store the binary representation, the first element is the less significative and the last is the most significative (reverse the original representation)
        binary_ids.append(binary_id)
    binary_ids.reverse()
    return binary_ids

def get_syn_dg(input_neuron_ids):
    # Get binary representation
    binary_ids = get_bin_representation(list(range(1, 2**len(input_neuron_ids))))
    pre_exc_syn = []
    post_exc_syn = []
    pre_inh_syn = []
    post_inh_syn = []
    # Get pre and post inh and exc synapses from binary representation
    for post, binary_id in enumerate(binary_ids):
        for pre, id in enumerate(binary_id):
            if id == 0:
                pre_inh_syn.append(pre)
                post_inh_syn.append(post)
            else:
                pre_exc_syn.append(pre)
                post_exc_syn.append(post)

    return pre_exc_syn, post_exc_syn, pre_inh_syn, post_inh_syn

def get_all_2_all_weighted(dg_neuron_ids, post_inh_syn):
    pre_dg_inh_syn = []
    post_dg_inh_syn = []
    # Get all to all conn
    for pre in range(len(dg_neuron_ids)):
        for post in range(pre+1, len(dg_neuron_ids)):
            pre_dg_inh_syn.append(pre)
            post_dg_inh_syn.append(post)
    # Add extra conn to special cases (dg neurons with no inhbitions)
    #   - Get what neurons are not inhibited
    neurons_no_inhibition = [i for i in range(len(dg_neuron_ids)) if not (i in post_inh_syn)]
    neurons_with_inhibition = [i for i in range(len(dg_neuron_ids)) if i in post_inh_syn]
    #   - For each one of them, add extra synapses
    for no_inh_neuron in neurons_no_inhibition:
        # One extra synapses for each neuron that have inhibition
        for inh_neuron in neurons_with_inhibition:
            pre_dg_inh_syn.append(inh_neuron)
            post_dg_inh_syn.append(no_inh_neuron)

    return pre_dg_inh_syn, post_dg_inh_syn

def get_syn_ca1(input_neuron_ids):
    # Get binary representation
    binary_ids = get_bin_representation(input_neuron_ids)
    pre_exc_syn = []
    post_exc_syn = []
    # Get pre and post exc synapses from binary representation
    for pre, binary_id in enumerate(binary_ids):
        for post, id in enumerate(binary_id):
            if id == 1:
                pre_exc_syn.append(pre)
                post_exc_syn.append(post)

    return pre_exc_syn, post_exc_syn

In [3455]:
# Initialization of all parameters
paramGroup = params.gen_clean_param_group()
for chip in range(4):
    for core in range(4):
        model.update_parameter_group(paramGroup, chip, core)

# Initialize STDP parameters
set_stdp_params(model)

In [3456]:
#       Hippocampus memory:
#           pattern lenght=11
#           cue = 4 -> 15 differents patterns
#           cont = 4 -> 4 bits of information

# Learning map (0) or system testing (1)
exp_id = 0

# Poisson generator frequency
poisson_freq = 0.5

# Noise
#   0 = nothing
#   1 = cue only
#   2 = cont only
#   3 = both
noise_target = 1

# Repetition id -> 1 - 5
rep_id = 5

if exp_id == 0:
    exp_name = "learn"
else:
    exp_name = "recall"

noise_target_name = "-"
if noise_target == 1:
    noise_target_name = "cue"
elif noise_target == 2:
    noise_target_name = "cont"
elif noise_target == 3:
    noise_target_name = "both"
elif noise_target == 0:
    noise_target_name = "not"

res_exp_name = "sequential_noise_1_A_noise"+noise_target_name+"_freq"+str(poisson_freq)+"_rep"+str(rep_id)+"_"+exp_name
path = "results/sequential_memory_noise/1/A/"+noise_target_name+"/"

if exp_id > 0:
    #initial_matrix_path = "results/sequential_memory_noise/1/without_noise/matrix_w_sequential_noise_0_2023_07_17_15_55_13.npy"
    initial_matrix_path = path+"matrix_w_sequential_noise_1_A_noise"+noise_target_name+"_freq"+str(poisson_freq)+"_rep"+str(rep_id)+"_learn"+".npy"

# NETWORK:
#   1) Create populations of neurons (chip id, core id, neurons id, generator?)
#       - Spike generator populations 
gen_cue_neuron_ids = [1, 2, 3, 4]
gen_cont_neuron_ids = [5, 6, 7, 8]
gen_cue_noise_neuron_ids = [9, 10, 11, 12]
gen_cont_noise_neuron_ids = [13, 14, 15, 16]
spikegen_cue_pop = n.NeuronGroup(0 ,0, gen_cue_neuron_ids, True) 
spikegen_cont_pop = n.NeuronGroup(0 ,0, gen_cont_neuron_ids, True)
spikegen_cue_noise_pop = n.NeuronGroup(0 ,0, gen_cue_noise_neuron_ids, True)
spikegen_cont_noise_pop = n.NeuronGroup(0 ,0, gen_cont_noise_neuron_ids, True)
#       - DPI neuron populations
#           * IN
in_pop_chip = 0
in_pop_core = 0
in_cue_neuron_ids = [233, 234, 218, 219]
in_cue_pop = n.NeuronGroup(in_pop_chip , in_pop_core, in_cue_neuron_ids, False)
in_cont_neuron_ids = [220, 221, 222, 223]
in_cont_pop = n.NeuronGroup(in_pop_chip , in_pop_core, in_cont_neuron_ids, False)
#           * NOISE IN
noise_cue_neuron_ids = [224, 232, 226, 227]
noise_cue_pop = n.NeuronGroup(in_pop_chip , in_pop_core, noise_cue_neuron_ids, False)
noise_cont_neuron_ids = [228, 229, 230, 231]
noise_cont_pop = n.NeuronGroup(in_pop_chip , in_pop_core, noise_cont_neuron_ids, False)
#           * DG
dg_pop_chip = 0
dg_pop_core = 1
delay_neurons_ids = [[1, 2, 3, 4], [194, 190, 7, 196], [195, 10, 11, 184], []] # neurons of Delay In pop in each layer
dg_neuron_ids = [[13, 14, 15, 16], [17, 18, 199, 20, 21, 22], [23, 24, 25, 26], [55]] # neurons in DG pop in each layer
delay_dg_neurons_ids = [[], [27, 28, 29, 30], [31, 32, 33, 34, 35, 36, 186, 38, 39, 40], [41, 42, 157, 48, 43, 215, 46, 49, 50, 52, 47, 158, 53, 54]] # neuron in Delay DG pop in each layer
dg_neurons_index = [[0, 1, 3, 7], [2, 4, 5, 8, 9, 11], [6, 10, 12, 13], [14]] # index of neurons in DG pops if they has not layers
delay_dg_neurons_id = [0, 1, 4, 2, 5, 6, 10, 3, 7, 8, 11, 9, 12, 13] # id (order) of neurons in the last population of delay dg -> only for stright-forward visual results
dg_all_neurons = [1, 2, 3, 4, 194, 190, 7, 196, 195, 10, 11, 184, 13, 14, 15, 16, 17, 18, 199, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 186, 38, 39, 40, 41, 42, 157, 48, 43, 215, 46, 49, 50, 52, 47, 158, 53, 54, 55]
dg_cont_neuron_ids = [56, 57, 58, 59]
num_dg_neurons = 55
#                   - DG
dg_pop = []
for neurons_id in dg_neuron_ids:
    if not(len(neurons_id) == 0):
        dg_pop.append(n.NeuronGroup(dg_pop_chip , dg_pop_core, neurons_id, False))
    else:
        dg_pop.append(False)
#                    - Delay
delay_pop = []
for neurons_id in delay_neurons_ids:
    if not(len(neurons_id) == 0):
        delay_pop.append(n.NeuronGroup(dg_pop_chip , dg_pop_core, neurons_id, False))
    else:
        delay_pop.append(False)
#                    - Delay DG
delay_dg_pop = []
for neurons_id in delay_dg_neurons_ids:
    if not(len(neurons_id) == 0):
        delay_dg_pop.append(n.NeuronGroup(dg_pop_chip , dg_pop_core, neurons_id, False))
    else:
        delay_dg_pop.append(False)
#                    - DG cont
dg_cont_pop = n.NeuronGroup(dg_pop_chip , dg_pop_core, dg_cont_neuron_ids, False)
#           * CA3 cue
ca3cue_pop_chip = 0
ca3cue_pop_core = 2
ca3cue_neuron_ids = [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 155, 71, 72, 73, 74]
ca3cue_pop = n.NeuronGroup(ca3cue_pop_chip , ca3cue_pop_core, ca3cue_neuron_ids, False)
#           * CA3 cont
ca3cont_pop_chip = 0
ca3cont_pop_core = 3
ca3cont_neuron_ids = [75, 76, 77, 78]
ca3cont_pop = n.NeuronGroup(ca3cont_pop_chip , ca3cont_pop_core, ca3cont_neuron_ids, False)
#           * CA1
ca1_pop_chip = 2
ca1_pop_core = 1
ca1_cue_neuron_ids = [79, 80, 81, 82]
ca1_cont_neuron_ids = [160, 84, 85, 86]
ca1_cue_pop = n.NeuronGroup(ca1_pop_chip , ca1_pop_core, ca1_cue_neuron_ids, False)
ca1_cont_pop = n.NeuronGroup(ca1_pop_chip , ca1_pop_core, ca1_cont_neuron_ids, False)
#           * EC (delay chain 7 layer/ 9 Dt)
ec_pop_chip = 2
ec_pop_core = 0
ec_neuron_ids = [[87, 88, 185, 90], [91, 92, 93, 94], [95, 163, 97, 98], [99, 191, 101, 102],
                  [151, 188, 105, 192], [107, 108, 109, 187], [111, 112, 152, 114], [115, 116, 117, 118],
                  [119, 167, 121, 122], [123, 124, 162, 164], [127, 193, 129, 165], [131, 153, 133, 134], 
                  [161, 136, 137, 159], [139, 140, 141, 142], [143, 144, 145, 146], [154, 148, 166, 150],
                  [168, 169, 170, 197], [172, 173, 174, 175], [176, 177, 178, 179], [180, 181, 182, 183],
                  [199, 200, 201, 202], [203, 204, 205, 206], [207, 208, 209, 210], [211, 212, 213, 214]]
# LAST NEURON ID USED = 240
ec_pop = []
for neurons_id in ec_neuron_ids:
    ec_pop.append(n.NeuronGroup(ec_pop_chip , ec_pop_core, neurons_id, False))
ec_all_neuron_ids = [x for pop in ec_pop for x in pop.neuron_ids]

# BE CAREFULL WITH THAT VALUE -> if stdp is crashing, reduce it
#max_fan_in_per_neuron = min(int(63/len(ca3cue_neuron_ids)), 5)
max_fan_in_per_neuron = 60

#   2) Create the network configuration for Dynapse (based on populations created and used to add synapses between them)
net_gen = n.NetworkGenerator()

#   3) Add synapses
#       - gen_cue-in -> excitatory and inhibitory
gen_in_syn = n.Synapses(spikegen_cue_pop, in_cue_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, gen_in_syn)
#       - gen_cont-in -> 1 to 1 excitatory
gencont_in_syn = n.Synapses(spikegen_cont_pop, in_cont_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, gencont_in_syn)
#       - gen_noise-in_noise -> 1 to 1 excitatory
if noise_target == 1 or noise_target == 3:
    gencue_noise_noise_syn = n.Synapses(spikegen_cue_noise_pop, noise_cue_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
    n.add_synapses(net_gen, gencue_noise_noise_syn)
if noise_target == 2 or noise_target == 3:
    gencont_noise_noise_syn = n.Synapses(spikegen_cont_noise_pop, noise_cont_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
    n.add_synapses(net_gen, gencont_noise_noise_syn)
#       - noise_in -> 1 to 1 excitatory
noise_cue_in_syn = n.Synapses(noise_cue_pop, in_cue_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, noise_cue_in_syn)
noise_cont_in_syn = n.Synapses(noise_cont_pop, in_cont_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, noise_cont_in_syn)
#       - in-dg
#           - Calculate the excitatory and inhibitory synapses
pre_exc_syn, post_exc_syn, pre_inh_syn, post_inh_syn = get_syn_dg(gen_cue_neuron_ids)
#           - IN-DELAY
in_delay_syn = n.Synapses(in_cue_pop, delay_pop[0], dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, in_delay_syn)
#           - DELAY-DELAY
delay_delay_syn = []
for i in range(1, len(gen_cue_neuron_ids)-1):
    if not(len(delay_neurons_ids[i]) == 0):
        delay_delay_syn.append(n.Synapses(delay_pop[i-1], delay_pop[i], dyn1.Dynapse1SynType.AMPA, conn_type='one2one'))
        n.add_synapses(net_gen, delay_delay_syn[len(delay_delay_syn)-1])
#           - IN/Delay - DG
out_neurons = 0
for neurons_ids in dg_neuron_ids:
    out_neurons = out_neurons + len(neurons_ids)
for i in range(len(dg_pop)): 
    #   - Search synapses for each neuron to add
    exc_syn_per_out_neuron = []
    inh_syn_per_out_neuron = []
    for out_neuron in range(out_neurons):
        if post_exc_syn.count(out_neuron) == i+1:
            # exc
            indices = [index for index, post_neuron in enumerate(post_exc_syn) if post_neuron == out_neuron]
            post_syn_exc = []
            pre_syn_exc = []
            for index in indices:
                post_syn_exc.append(dg_neurons_index[i].index(post_exc_syn[index]))
                pre_syn_exc.append(pre_exc_syn[index])
            exc_syn_per_out_neuron.append([pre_syn_exc, post_syn_exc])
            # inh
            indices = [index for index, post_neuron in enumerate(post_inh_syn) if post_neuron == out_neuron]
            post_syn_inh = []
            pre_syn_inh = []
            for index in indices:
                post_syn_inh.append(dg_neurons_index[i].index(post_inh_syn[index]))
                pre_syn_inh.append(pre_inh_syn[index])
            inh_syn_per_out_neuron.append([pre_syn_inh, post_syn_inh])
    #   - Add synapses
    #       * Exc
    for neuron_syn_index in exc_syn_per_out_neuron:
        if i == 0:
            net_gen.add_connections_from_list(in_cue_pop.neurons, dg_pop[i].neurons, dyn1.Dynapse1SynType.AMPA, pre_ids = neuron_syn_index[0], post_ids = neuron_syn_index[1])
        else:
            net_gen.add_connections_from_list(delay_pop[i-1].neurons, dg_pop[i].neurons, dyn1.Dynapse1SynType.AMPA, pre_ids = neuron_syn_index[0], post_ids = neuron_syn_index[1])
    #       * Inh
    for neuron_syn_index in inh_syn_per_out_neuron:
        if i == 0:
            net_gen.add_connections_from_list(in_cue_pop.neurons, dg_pop[i].neurons, dyn1.Dynapse1SynType.GABA_B, pre_ids = neuron_syn_index[0], post_ids = neuron_syn_index[1])
        else:
            net_gen.add_connections_from_list(delay_pop[i-1].neurons, dg_pop[i].neurons, dyn1.Dynapse1SynType.GABA_B, pre_ids = neuron_syn_index[0], post_ids = neuron_syn_index[1])
#           - DG(layer i-1)-DG(layer i) inh
for i in range(len(dg_neuron_ids)):
    if not(i == 0):
        dg_dg_inh_syn = n.Synapses(dg_pop[i-1], dg_pop[i], dyn1.Dynapse1SynType.GABA_B, conn_type='all2all')
        n.add_synapses(net_gen, dg_dg_inh_syn)
for i in range(len(delay_dg_neurons_ids)):
    if not(i == 0) and not(type(delay_dg_pop[i-1]) is bool):
        dg_dg_inh_syn = n.Synapses(delay_dg_pop[i-1], dg_pop[i], dyn1.Dynapse1SynType.GABA_B, conn_type='all2all')
        n.add_synapses(net_gen, dg_dg_inh_syn)    
#           - DG(layer i-1)-DG delay(layer i) exc
#           - DG delay(layer i-1)-DG delay(layer i) exc
for i in range(len(dg_neuron_ids)):
    if not(i == 0) and not(type(delay_dg_pop[i-1]) is bool):
        net_gen.add_connections_from_list(delay_dg_pop[i-1].neurons, delay_dg_pop[i].neurons, dyn1.Dynapse1SynType.AMPA, 
                                        pre_ids = list(range(len(delay_dg_neurons_ids[i-1]))), 
                                        post_ids = list(range(len(delay_dg_neurons_ids[i-1]))))
    if not(i == 0):
        net_gen.add_connections_from_list(dg_pop[i-1].neurons, delay_dg_pop[i].neurons, dyn1.Dynapse1SynType.AMPA, 
                                        pre_ids = list(range(len(dg_neuron_ids[i-1]))), 
                                        post_ids = list(range(len(delay_dg_neurons_ids[i-1]), len(delay_dg_neurons_ids[i-1]) + len(dg_neuron_ids[i-1]))))

#       - in-dg -> 1 to 1 excitatory
gencont_dg_syn = n.Synapses(in_cont_pop, dg_cont_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, gencont_dg_syn)
#       - dg-cue -> 1 to 1 excitatory
net_gen.add_connections_from_list(delay_dg_pop[len(delay_dg_pop)-1].neurons, ca3cue_pop.neurons, dyn1.Dynapse1SynType.AMPA, 
                                    pre_ids = delay_dg_neurons_id, 
                                    post_ids = list(range(len(delay_dg_neurons_ids[len(delay_dg_neurons_ids)-1]))))
net_gen.add_connections_from_list(dg_pop[len(dg_pop)-1].neurons, ca3cue_pop.neurons, dyn1.Dynapse1SynType.AMPA, 
                                    pre_ids = [0], 
                                    post_ids = [len(ca3cue_neuron_ids)-1])
#       - dg-cont -> 1 to 1 excitatory
dg_ca3cont_syn = n.Synapses(dg_cont_pop, ca3cont_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, dg_ca3cont_syn)
#       - ca3cue-ca3cont -> all to all excitatory (STDP)
if exp_id == 0:
    # Initial case
    initial_w = 0.0
    initial_w_matrix = np.ones((len(ca3cue_pop.neurons), len(ca3cont_pop.neurons)))*initial_w
    initial_int_w_plast = initial_w_matrix
elif exp_id == 1:
    # Taking previous map learned
    initial_w_matrix = np.load(initial_matrix_path)
    initial_int_w_plast = stdp_utils.floatW2intW(initial_w_matrix, max_fan_in_per_neuron)

initial_w_matrix_copy = initial_w_matrix.copy()
ca3cue_ca3cont_syn = n.Synapses(ca3cue_pop, ca3cont_pop, dyn1.Dynapse1SynType.NMDA, weight_matrix=initial_int_w_plast)
n.add_synapses(net_gen, ca3cue_ca3cont_syn)
#       - ca3cue-ca1 -> excitatory
pre_exc_syn, post_exc_syn = get_syn_ca1(list(range(1, len(ca3cue_neuron_ids)+1)))
ca3cue_ca1_syn = net_gen.add_connections_from_list(ca3cue_pop.neurons, ca1_cue_pop.neurons, dyn1.Dynapse1SynType.AMPA, pre_ids = pre_exc_syn, post_ids = post_exc_syn)
#       - ca3cont-ca1 -> 1 to 1 excitatory
ca3cont_ca1_syn = n.Synapses(ca3cont_pop, ca1_cont_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, ca3cont_ca1_syn)

#       - CA1cont - EC(1) -> 1 to 1 excitatory
if exp_id == 1:
    ca1_ec_syn = n.Synapses(ca1_cont_pop, ec_pop[0], dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
    n.add_synapses(net_gen, ca1_ec_syn)
#       - EC(i) - EC(i+1) -> 1 to 1 excitatory
for i in range(1, len(ec_pop)):
    ec_ec_syn = n.Synapses(ec_pop[i-1], ec_pop[i], dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
    n.add_synapses(net_gen, ec_ec_syn)
#       - EC(last) - in:
ec_in_syn = n.Synapses(ec_pop[i-1], in_cue_pop, dyn1.Dynapse1SynType.AMPA, conn_type='one2one')
n.add_synapses(net_gen, ec_in_syn)

In [3457]:
print("- Network: ")
print(net_gen.network)

- Network: 
Post neuron (ChipId,coreId,neuronId): incoming connections [(preNeuron,synapseType), ...]
C0c0n233: [('C0c0s1', 'AMPA'), ('C0c0n224', 'AMPA'), ('C2c0n207', 'AMPA')]
C0c0n234: [('C0c0s2', 'AMPA'), ('C0c0n232', 'AMPA'), ('C2c0n208', 'AMPA')]
C0c0n218: [('C0c0s3', 'AMPA'), ('C0c0n226', 'AMPA'), ('C2c0n209', 'AMPA')]
C0c0n219: [('C0c0s4', 'AMPA'), ('C0c0n227', 'AMPA'), ('C2c0n210', 'AMPA')]
C0c0n220: [('C0c0s5', 'AMPA'), ('C0c0n228', 'AMPA')]
C0c0n221: [('C0c0s6', 'AMPA'), ('C0c0n229', 'AMPA')]
C0c0n222: [('C0c0s7', 'AMPA'), ('C0c0n230', 'AMPA')]
C0c0n223: [('C0c0s8', 'AMPA'), ('C0c0n231', 'AMPA')]
C0c0n224: [('C0c0s9', 'AMPA')]
C0c0n232: [('C0c0s10', 'AMPA')]
C0c0n226: [('C0c0s11', 'AMPA')]
C0c0n227: [('C0c0s12', 'AMPA')]
C0c0n228: [('C0c0s13', 'AMPA')]
C0c0n229: [('C0c0s14', 'AMPA')]
C0c0n230: [('C0c0s15', 'AMPA')]
C0c0n231: [('C0c0s16', 'AMPA')]
C0c1n1: [('C0c0n233', 'AMPA')]
C0c1n2: [('C0c0n234', 'AMPA')]
C0c1n3: [('C0c0n218', 'AMPA')]
C0c1n4: [('C0c0n219', 'AMPA')]
C0c1n19

In [3458]:
# Create configuration based on network created (if not valid raise an exception and close the device)
config = net_gen.make_dynapse1_configuration()

# Apply configuration
model.apply_configuration(config)

In [3459]:
# Instantiate the STDP algorithm
#   * Model
#   * Network generated
#   * Presynaptic neurons
ca3cue_neurons = [(ca3cue_pop_chip, ca3cue_pop_core, x) for x in ca3cue_neuron_ids]
#   * Postsynaptic neurons 
ca3cont_neurons = [(ca3cont_pop_chip, ca3cont_pop_core, x) for x in ca3cont_neuron_ids]
#   * Weight matrix
#   * STDP parameters file path
#   * STDP algorithm (only triplet_stdp)
#   * If manage the STDP in a new thread
stdp = Stdp(model, net_gen, ca3cue_neurons, ca3cont_neurons, initial_w_matrix, 'triplet_stdp_params_sequential_noise.json', algorithm='triplet_stdp', new_thread=True)

In [3460]:
# Initialize custom parameters for a core and a chip:
paramGroup = dyn1.Dynapse1ParameterGroup() 

# IN
paramGroup = dyn1.Dynapse1ParameterGroup() 
#       - Neuron:
#           * THR (IF_THR_N): the gain factor of neuron
paramGroup.param_map["IF_THR_N"].coarse_value = 4
paramGroup.param_map["IF_THR_N"].fine_value = 120
#           * RFR (IF_RFR_N): refractory period
paramGroup.param_map["IF_RFR_N"].coarse_value = 7
paramGroup.param_map["IF_RFR_N"].fine_value = 255
#           * TAU1 (IF_TAU1_N) and TAU2 (IF_TAU2_N): the membrane time-constant of leakage current or how much current is constantly leaked away
paramGroup.param_map["IF_TAU1_N"].coarse_value = 2
paramGroup.param_map["IF_TAU1_N"].fine_value = 5
#           * DC (IF_DC_P): injection of constant current
paramGroup.param_map["IF_DC_P"].coarse_value = 0
paramGroup.param_map["IF_DC_P"].fine_value = 0
#       - Synapses:
#           - AMPA (IE_F_P): Fast excitatory synapses
#               * TAU (NPDPIE_TAU_F_P or NPDPII_TAU_F_P)
paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value =  80
#               * THR (NPDPIE_THR_F_P or NPDPII_THR_F_P)
paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
paramGroup.param_map["NPDPIE_THR_F_P"].fine_value =  80 
#               * WEIGHT (PS_WEIGHT_EXC_F_N or PS_WEIGHT_INH_F_N)
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 255 
model.update_parameter_group(paramGroup, in_cue_pop.chip_id, in_cue_pop.core_id)

# DG
#       - Neuron:
#           * THR (IF_THR_N): the gain factor of neuron
paramGroup.param_map["IF_THR_N"].coarse_value = 4
paramGroup.param_map["IF_THR_N"].fine_value = 150
#           * RFR (IF_RFR_N): refractory period
paramGroup.param_map["IF_RFR_N"].coarse_value = 7
paramGroup.param_map["IF_RFR_N"].fine_value = 255
#           * TAU1 (IF_TAU1_N) and TAU2 (IF_TAU2_N): the membrane time-constant of leakage current or how much current is constantly leaked away
paramGroup.param_map["IF_TAU1_N"].coarse_value = 2
paramGroup.param_map["IF_TAU1_N"].fine_value = 150
#           * DC (IF_DC_P): injection of constant current
paramGroup.param_map["IF_DC_P"].coarse_value = 0
paramGroup.param_map["IF_DC_P"].fine_value = 0
#       - Synapses:
#           - AMPA (IE_F_P): Fast excitatory synapses
#               * TAU (NPDPIE_TAU_F_P or NPDPII_TAU_F_P)
paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value =  80
#               * THR (NPDPIE_THR_F_P or NPDPII_THR_F_P)
paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 4
paramGroup.param_map["NPDPIE_THR_F_P"].fine_value =  80 
#               * WEIGHT (PS_WEIGHT_EXC_F_N or PS_WEIGHT_INH_F_N)
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 255 
#           - GABA_B
paramGroup.param_map["NPDPII_TAU_S_P"].coarse_value = 2
paramGroup.param_map["NPDPII_TAU_S_P"].fine_value =  128 
paramGroup.param_map["NPDPII_THR_S_P"].coarse_value = 7
paramGroup.param_map["NPDPII_THR_S_P"].fine_value =  200
paramGroup.param_map["PS_WEIGHT_INH_S_N"].coarse_value = 7
paramGroup.param_map["PS_WEIGHT_INH_S_N"].fine_value = 255 

# Apply the custom parameters to the core
model.update_parameter_group(paramGroup, dg_pop[0].chip_id, dg_pop[0].core_id)


# CA3CUE
paramGroup = dyn1.Dynapse1ParameterGroup() 
#       - Neuron:
#           * THR (IF_THR_N): the gain factor of neuron
paramGroup.param_map["IF_THR_N"].coarse_value = 5
paramGroup.param_map["IF_THR_N"].fine_value = 100
#           * RFR (IF_RFR_N): refractory period
paramGroup.param_map["IF_RFR_N"].coarse_value = 4
paramGroup.param_map["IF_RFR_N"].fine_value = 120
#           * TAU1 (IF_TAU1_N) and TAU2 (IF_TAU2_N): the membrane time-constant of leakage current or how much current is constantly leaked away
paramGroup.param_map["IF_TAU1_N"].coarse_value = 3
paramGroup.param_map["IF_TAU1_N"].fine_value = 15
#           * DC (IF_DC_P): injection of constant current
paramGroup.param_map["IF_DC_P"].coarse_value = 0
paramGroup.param_map["IF_DC_P"].fine_value = 0
#       - Synapses:
#           - AMPA (IE_F_P): Fast excitatory synapses
#           - GABA_A (II_F_P): Fast inhibitory synapses
#               * TAU (NPDPIE_TAU_F_P or NPDPII_TAU_F_P)
paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value =  80 
#               * THR (NPDPIE_THR_F_P or NPDPII_THR_F_P)
paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 4
paramGroup.param_map["NPDPIE_THR_F_P"].fine_value =  80 
#               * WEIGHT (PS_WEIGHT_EXC_F_N or PS_WEIGHT_INH_F_N)
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 255 
#           - NMDA (S_P): slow excitatory synapses
#               * THRESHOLD (IF_NMDA_N)
paramGroup.param_map["IF_NMDA_N"].coarse_value = 0
paramGroup.param_map["IF_NMDA_N"].fine_value = 0
paramGroup.param_map["PS_WEIGHT_EXC_S_N"].coarse_value = 0
paramGroup.param_map["PS_WEIGHT_EXC_S_N"].fine_value = 0 
model.update_parameter_group(paramGroup, ca3cue_pop.chip_id, ca3cue_pop.core_id)

# CA3CONT
paramGroup.param_map["IF_THR_N"].coarse_value = 5
paramGroup.param_map["IF_THR_N"].fine_value = 64
paramGroup.param_map["IF_RFR_N"].coarse_value = 4
paramGroup.param_map["IF_RFR_N"].fine_value = 120
paramGroup.param_map["IF_TAU1_N"].coarse_value = 3
paramGroup.param_map["IF_TAU1_N"].fine_value = 60

paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 255 
model.update_parameter_group(paramGroup, ca3cont_pop.chip_id, ca3cont_pop.core_id)


# CA1
paramGroup = dyn1.Dynapse1ParameterGroup() 
#       - Neuron:
#           * THR (IF_THR_N): the gain factor of neuron
paramGroup.param_map["IF_THR_N"].coarse_value = 5
paramGroup.param_map["IF_THR_N"].fine_value = 100
#           * RFR (IF_RFR_N): refractory period
paramGroup.param_map["IF_RFR_N"].coarse_value = 7
paramGroup.param_map["IF_RFR_N"].fine_value = 255
#           * TAU1 (IF_TAU1_N) and TAU2 (IF_TAU2_N): the membrane time-constant of leakage current or how much current is constantly leaked away
paramGroup.param_map["IF_TAU1_N"].coarse_value = 2
paramGroup.param_map["IF_TAU1_N"].fine_value = 150
#           * DC (IF_DC_P): injection of constant current
paramGroup.param_map["IF_DC_P"].coarse_value = 0
paramGroup.param_map["IF_DC_P"].fine_value = 0
#       - Synapses:
#           - AMPA (IE_F_P): Fast excitatory synapses
#               * TAU (NPDPIE_TAU_F_P or NPDPII_TAU_F_P)
paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value =  80
#               * THR (NPDPIE_THR_F_P or NPDPII_THR_F_P)
paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
paramGroup.param_map["NPDPIE_THR_F_P"].fine_value =  80 
#               * WEIGHT (PS_WEIGHT_EXC_F_N or PS_WEIGHT_INH_F_N)
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 255 
model.update_parameter_group(paramGroup, ca1_cue_pop.chip_id, ca1_cue_pop.core_id)

# EC
paramGroup = dyn1.Dynapse1ParameterGroup() 
#       - Neuron:
#           * THR (IF_THR_N): the gain factor of neuron
paramGroup.param_map["IF_THR_N"].coarse_value = 5
paramGroup.param_map["IF_THR_N"].fine_value = 120
#           * RFR (IF_RFR_N): refractory period
paramGroup.param_map["IF_RFR_N"].coarse_value = 7
paramGroup.param_map["IF_RFR_N"].fine_value = 255
#           * TAU1 (IF_TAU1_N) and TAU2 (IF_TAU2_N): the membrane time-constant of leakage current or how much current is constantly leaked away
paramGroup.param_map["IF_TAU1_N"].coarse_value = 2
paramGroup.param_map["IF_TAU1_N"].fine_value = 5
#           * DC (IF_DC_P): injection of constant current
paramGroup.param_map["IF_DC_P"].coarse_value = 0
paramGroup.param_map["IF_DC_P"].fine_value = 0
#       - Synapses:
#           - AMPA (IE_F_P): Fast excitatory synapses
#               * TAU (NPDPIE_TAU_F_P or NPDPII_TAU_F_P)
paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value =  80
#               * THR (NPDPIE_THR_F_P or NPDPII_THR_F_P)
paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
paramGroup.param_map["NPDPIE_THR_F_P"].fine_value =  80 
#               * WEIGHT (PS_WEIGHT_EXC_F_N or PS_WEIGHT_INH_F_N)
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 255 
model.update_parameter_group(paramGroup, ec_pop[0].chip_id, ec_pop[0].core_id)

In [3461]:
"""
Experiments:
    0 - learn the map
    1 - recall the path to the exit from all known positions with some induce noise
"""
experiment = exp_id

# Create the generator of spikes: set_fpga_spike_gen(*)
#   * generator object
fpga_spike_gen = model.get_fpga_spike_gen()
#   * list of input spikes in seconds = spike list
if experiment == 0:
    spike_train_time = 0.01
    spike_train_num = 25
    time_to_last_spike = 0.01
    total_time_sample = 0.02
    time_between_samples = 0.1
    num_samples = 3
    #   * Learn
    #       [1] = [1] = 2b10 = (2)6 -> 2
    #       [2] = [2] = -
    #       [3] = [1, 2] = 2b10 = (2)6 -> 3
    #       [4] = [3] = 3b10 = (1, 2)5, 6 -> 3
    #       [5] = [1, 3] = 1b10 = (1)5 -> 3
    #       [6] = [2, 3] = 2b10 = (2)6 -> 3
    #       [7] = [1, 2, 3] = 6b10 = (2, 3)6, 7 -> 5
    #       [8] = [4] = 4b10 = (3)7 -> 2
    #       [9] = [1, 4] = 10b10 = (2, 4)6, 8 -> 4
    #       [10] = [2, 4] = 6b10 = (2, 3)6, 7 -> 4
    #       [11] = [1, 2, 4] = 7b10 = (1, 2, 3)5, 6, 7 -> 6
    #       [12] = [3, 4] = 11b10 = (1, 2, 4)5, 6, 8 -> 5
    #       [13] = [1, 3, 4] = 9b10 = (1, 4)5, 8 -> 5
    #       [14] = [2, 3, 4] = 10b10 = (2, 4)6, 8 -> 5
    #       [15] = [1, 2, 3, 4] = 14b10 = (2, 3, 4)6, 7, 8 -> 7
    spike_time_list = [np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                              axis=None), axis=None)] 
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    operation_samples = [num_samples] * len(spike_time_list)  
    ca3cue_ca3cont_comb = [[[6], [], [], [], [], [], [], [], [], [], [], [], [], [], []]]
    ca3cue_ca3cont_comb.append([[], [], [6], [], [], [], [], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [5, 6], [], [], [], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [5], [], [], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [6], [], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [6, 7], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [7], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [6, 8], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [6, 7], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [5, 6, 7], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [], [5, 6, 8], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [], [], [5, 8], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [], [], [], [6, 8], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [], [], [], [], [6, 7, 8]])
elif experiment == 1:
    spike_train_time = 0.01
    spike_train_num = 25
    time_to_last_spike = 0.01
    total_time_sample = 0.05
    time_between_samples = 0.3
    num_samples = 3
    #   * Recall
    #       [1] = [1]
    #       [3] = [1, 2] 
    #       [4] = [3] 
    #       [5] = [1, 3] 
    #       [6] = [2, 3]
    #       [7] = [1, 2, 3] 
    #       [8] = [4] 
    #       [9] = [1, 4]
    #       [10] = [2, 4]
    #       [11] = [1, 2, 4]
    #       [12] = [3, 4]
    #       [13] = [1, 3, 4]
    #       [14] = [2, 3, 4]
    #       [15] = [1, 2, 3, 4]
    spike_time_list = [np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num)), 
                                              axis=None), axis=None)] 
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    spike_time_list.append(np.sort(np.concatenate((np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num),
                                                   np.linspace(0, spike_train_time, spike_train_num), np.linspace(0, spike_train_time, spike_train_num)), 
                                                  axis=None), axis=None))
    operation_samples = [1] * len(spike_time_list)  
    ca3cue_ca3cont_comb = [[[6], [], [], [], [], [], [], [], [], [], [], [], [], [], []]]
    ca3cue_ca3cont_comb.append([[], [], [6], [], [], [], [], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [5, 6], [], [], [], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [5], [], [], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [6], [], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [6, 7], [], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [7], [], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [6, 8], [], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [6, 7], [], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [5, 6, 7], [], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [], [5, 6, 8], [], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [], [], [5, 8], [], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [], [], [], [6, 8], []])
    ca3cue_ca3cont_comb.append([[], [], [], [], [], [], [], [], [], [], [], [], [], [], [6, 7, 8]])
else:
    spike_time_list = []
    time_to_last_spike = 0
#   * list of generator neuron ids (one element per spike in spike list)
if experiment == 0:
    # Learn
    gen_neuron_ids = [[1, 6]*(int(len(spike_time_list[0])/2))]
    gen_neuron_ids.append([1, 2, 6]*(int(len(spike_time_list[1])/3)))
    gen_neuron_ids.append([3, 5, 6]*(int(len(spike_time_list[2])/3)))
    gen_neuron_ids.append([1, 3, 5]*(int(len(spike_time_list[3])/3)))
    gen_neuron_ids.append([2, 3, 6]*(int(len(spike_time_list[4])/3)))
    gen_neuron_ids.append([1, 2, 3, 6, 7]*(int(len(spike_time_list[5])/5)))
    gen_neuron_ids.append([4, 7]*(int(len(spike_time_list[6])/2)))
    gen_neuron_ids.append([1, 4, 6, 8]*(int(len(spike_time_list[7])/4)))
    gen_neuron_ids.append([2, 4, 6, 7]*(int(len(spike_time_list[8])/4)))
    gen_neuron_ids.append([1, 2, 4, 5, 6, 7]*(int(len(spike_time_list[9])/6)))
    gen_neuron_ids.append([3, 4, 5, 6, 8]*(int(len(spike_time_list[10])/5)))
    gen_neuron_ids.append([1, 3, 4, 5, 8]*(int(len(spike_time_list[11])/5)))
    gen_neuron_ids.append([2, 3, 4, 6, 8]*(int(len(spike_time_list[12])/5)))
    gen_neuron_ids.append([1, 2, 3, 4, 6, 7, 8]*(int(len(spike_time_list[13])/7)))
elif experiment == 1:
    # Recall
    gen_neuron_ids = [[1]*(int(len(spike_time_list[0])))]
    gen_neuron_ids.append([1, 2]*(int(len(spike_time_list[1])/2)))
    gen_neuron_ids.append([3]*(int(len(spike_time_list[2]))))
    gen_neuron_ids.append([1, 3]*(int(len(spike_time_list[3])/2)))
    gen_neuron_ids.append([2, 3]*(int(len(spike_time_list[4])/2)))
    gen_neuron_ids.append([1, 2, 3]*(int(len(spike_time_list[5])/3)))
    gen_neuron_ids.append([4]*(int(len(spike_time_list[6]))))
    gen_neuron_ids.append([1, 4]*(int(len(spike_time_list[7])/2)))
    gen_neuron_ids.append([2, 4]*(int(len(spike_time_list[8])/2)))
    gen_neuron_ids.append([1, 2, 4]*(int(len(spike_time_list[9])/3)))
    gen_neuron_ids.append([3, 4]*(int(len(spike_time_list[10])/2)))
    gen_neuron_ids.append([1, 3, 4]*(int(len(spike_time_list[11])/3)))
    gen_neuron_ids.append([2, 3, 4]*(int(len(spike_time_list[12])/3)))
    gen_neuron_ids.append([1, 2, 3, 4]*(int(len(spike_time_list[13])/4)))
else:
    gen_neuron_ids = []
#   * list of chips where the post neurons are (one element per spike in spike list)
if experiment >= 0:
    post_neuron_chips = [[0]*len(spike_time_list[0])]
    for i in range(1, len(spike_time_list)):
        post_neuron_chips.append([0]*len(spike_time_list[i]))
else:
    post_neuron_chips = []
#   * isi_base = 90 or 900, scaled the inter spike interval for 1 or 10 us
isi_base = 900
#   * repeat_mode = loop on the spikes sends when finish the list
repeat_mode = False

In [3462]:
print(len(spike_time_list))
print(len(spike_time_list[0]))
print(len(gen_neuron_ids))
print(len(gen_neuron_ids[0]))

14
25
14
25


In [3463]:
# Create poisson generators
if exp_id == 0:
    poisson_gens = []
    if noise_target == 1 or noise_target == 3:
        for i in range(len(gen_cue_noise_neuron_ids)):
            #   + Get poissongen from the model
            poisson_gens.append(model.get_poisson_gen())
            #   + Set the target chip of the post neurons
            poisson_gens[i].set_chip_id(in_pop_chip)
            #   + Set rate for each
            poisson_gens[i].write_poisson_rate_hz(gen_cue_noise_neuron_ids[i], poisson_freq)
    if noise_target == 2 or noise_target == 3:
        for i in range(len(gen_cont_noise_neuron_ids)):
            #   + Get poissongen from the model
            poisson_gens.append(model.get_poisson_gen())
            #   + Set the target chip of the post neurons
            poisson_gens[i].set_chip_id(in_pop_chip)
            #   + Set rate for each
            poisson_gens[i].write_poisson_rate_hz(gen_cont_noise_neuron_ids[i], poisson_freq)

In [3464]:
model.get_dynapse1_api().reset_timestamp()
if experiment >= 0:
    # Prepare to record spike activity of neurons
    monitored_neurons = []
    for pop in dg_pop:
        if not(type(pop) is bool):
            monitored_neurons = monitored_neurons + pop.tuple_neuron_ids
    for pop in delay_pop:
        if not(type(pop) is bool):
            monitored_neurons = monitored_neurons + pop.tuple_neuron_ids
    for pop in delay_dg_pop:
        if not(type(pop) is bool):
            monitored_neurons = monitored_neurons + pop.tuple_neuron_ids
    monitored_neurons = monitored_neurons + dg_cont_pop.tuple_neuron_ids
    monitored_neurons = monitored_neurons + ca3cue_pop.tuple_neuron_ids + ca3cont_pop.tuple_neuron_ids
    monitored_neurons = monitored_neurons + ca1_cue_pop.tuple_neuron_ids + ca1_cont_pop.tuple_neuron_ids
    monitored_neurons = monitored_neurons + in_cue_pop.tuple_neuron_ids + in_cont_pop.tuple_neuron_ids
    monitored_neurons = monitored_neurons + noise_cue_pop.tuple_neuron_ids + noise_cont_pop.tuple_neuron_ids

    #monitored_neurons = monitored_neurons + ec_pop[len(ec_pop)-1].tuple_neuron_ids
    for pop in ec_pop:
        monitored_neurons = monitored_neurons + pop.tuple_neuron_ids
    
    graph, filter_node, sink_node = ut.create_neuron_select_graph(model, monitored_neurons)
    graph.start()
    sink_node.get_events()
    # Start poisson generators
    if exp_id == 0:
        for i in range(len(poisson_gens)):
            poisson_gens[i].start()

    # Start stdp
    if exp_id == 0:
        stdp.start_stdp()
    # Learning separate for each pattern
    trace = []
    temporal_cost_per_sample = []
    first_iter = True
    for pattern in range(len(spike_time_list)):
        if operation_samples[pattern] == 3:
            print("LEARN " + str(pattern))
        else:
            print("RECALL " + str(pattern))
        # For each pattern, assign the generator activity
        ut.set_fpga_spike_gen(fpga_spike_gen, spike_time_list[pattern], gen_neuron_ids[pattern], post_neuron_chips[pattern], isi_base, repeat_mode)
        # Each pattern as a number of samples to the network
        trace_per_pattern = []
        for sample in range(operation_samples[pattern]):
            if first_iter:
                first_iter = False
            else:
                temporal_cost_per_sample.append(time.time() - start_sample)
            start_sample = time.time()
            # Start input spike
            fpga_spike_gen.start()
            time.sleep(time_to_last_spike)
            fpga_spike_gen.stop()
            time.sleep(total_time_sample-time_to_last_spike)
            
            if exp_id == 0:
                # Delete synapses
                n.remove_synapses(net_gen, ca3cue_ca3cont_syn)

                # Add new synapses with stdp weight change
                current_w_plast = stdp.w_plast
                int_w_plast = stdp_utils.floatW2intW(current_w_plast, max_fan_in_per_neuron)

                ca3cue_ca3cont_syn = n.Synapses(ca3cue_pop, ca3cont_pop, dyn1.Dynapse1SynType.NMDA, weight_matrix=int_w_plast)
                n.add_synapses(net_gen, ca3cue_ca3cont_syn)

                # Debug
                debug_matrix = current_w_plast.copy()
                trace_per_pattern.append(debug_matrix)

            if exp_id == 0:
                new_config = net_gen.make_dynapse1_configuration()
                model.apply_configuration(new_config)

            # cooling down
            time.sleep(time_between_samples)
        trace.append(trace_per_pattern)
    temporal_cost_per_sample.append(time.time() - start_sample)

    # Stop stdp, poisson, get events and stop graph
    if exp_id == 0:
        stdp.stop_stdp()
        for i in range(len(poisson_gens)):
            poisson_gens[i].stop()
    events = sink_node.get_events()
    graph.stop()

RECALL 0
RECALL 1
RECALL 2
RECALL 3
RECALL 4
RECALL 5
RECALL 6
RECALL 7
RECALL 8
RECALL 9
RECALL 10
RECALL 11
RECALL 12
RECALL 13


In [3465]:
if experiment == 0:
    results = ""
    # Generate and store the trace in a file
    #   Initial 
    results = results + "* Initial: \n"
    for pre in range(len(ca3cue_neuron_ids)):
        for post in range(len(ca3cont_neuron_ids)):
            results = results + "     (-)pre"+ str(pre) +"-post"+ str(post) +" = " + str(initial_w_matrix_copy[pre][post]) + "\n"
    #   Operations
    for pattern in range(len(spike_time_list)):
        if operation_samples[pattern] == 3:
            results = results + "LEARN " + str(pattern) + "\n"
        else:
            results = results + "RECALL " + str(pattern) + "\n"
        for sample in range(operation_samples[pattern]):
            results = results + "* Iter " + str(sample) + "\n"
            for pre in range(len(ca3cue_neuron_ids)):
                for post in range(len(ca3cont_neuron_ids)):
                    if post+len(gen_cue_neuron_ids)+1 in ca3cue_ca3cont_comb[pattern][pre]:
                        results = results + "     (+)pre"+ str(pre) +"-post"+ str(post) +" = " + str(trace[pattern][sample][pre][post]) + "\n"
                    else:
                        results = results + "     (-)pre"+ str(pre) +"-post"+ str(post) +" = " + str(trace[pattern][sample][pre][post]) + "\n"
    #   Save in file
    #file = open("results/sequential_memory_noise/1/without_noise/trace_sequential_noise_"+str(experiment)+"_"+str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))+".txt", "w")
    file = open(path+"trace_"+res_exp_name+".txt", "w")
    file.write(results)
    file.close()

    #np.save("results/sequential_memory_noise/1/without_noise/matrix_w_sequential_noise_"+str(experiment)+"_"+str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))+".npy", stdp.w_plast)
    np.save(path+"matrix_w_"+res_exp_name+".npy", stdp.w_plast)

In [3466]:
# All y_ticks
all_fullname_yticks = []
all_id_only_yticks = []
y_pop_separation = []
#   * NOISE
for neuron_id in range(len(noise_cue_neuron_ids)):
    all_fullname_yticks.append("NOISE_"+str(neuron_id))
    all_id_only_yticks.append(str(neuron_id))
for neuron_id in range(len(noise_cont_neuron_ids)):
    all_fullname_yticks.append("NOISE_"+str(neuron_id+len(noise_cue_neuron_ids)))
    all_id_only_yticks.append(str(neuron_id+len(noise_cue_neuron_ids)))
y_pop_separation.append(len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)-1)
#   * IN
for neuron_id in range(len(gen_cue_neuron_ids)):
    all_fullname_yticks.append("IN_"+str(neuron_id))
    all_id_only_yticks.append(str(neuron_id))
for neuron_id in range(len(gen_cont_neuron_ids)):
    all_fullname_yticks.append("IN_"+str(neuron_id+len(gen_cue_neuron_ids)))
    all_id_only_yticks.append(str(neuron_id+len(gen_cue_neuron_ids)))
y_pop_separation.append(len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + y_pop_separation[len(y_pop_separation)-1])
#   * DG
for neuron_id in range(len(dg_all_neurons)):
    all_fullname_yticks.append("DG_"+str(neuron_id))
    all_id_only_yticks.append(str(neuron_id))
for neuron_id in range(len(dg_cont_neuron_ids)):
    all_fullname_yticks.append("DG_"+str(neuron_id+len(dg_all_neurons)))
    all_id_only_yticks.append(str(neuron_id+len(dg_all_neurons)))
y_pop_separation.append(len(dg_all_neurons) + len(dg_cont_neuron_ids) + y_pop_separation[len(y_pop_separation)-1])
#   * CA3
for neuron_id in range(len(ca3cue_neuron_ids)):
    all_fullname_yticks.append("CA3cue_"+str(neuron_id))
    all_id_only_yticks.append(str(neuron_id))
for neuron_id in range(len(ca3cont_neuron_ids)):
    all_fullname_yticks.append("CA3cont_"+str(neuron_id))
    all_id_only_yticks.append(str(neuron_id))
y_pop_separation.append(len(ca3cue_neuron_ids) + y_pop_separation[len(y_pop_separation)-1])
y_pop_separation.append(len(ca3cont_neuron_ids) + y_pop_separation[len(y_pop_separation)-1])
#   * CA1
for neuron_id in range(len(ca1_cue_neuron_ids)):
    all_fullname_yticks.append("CA1_"+str(neuron_id))
    all_id_only_yticks.append(str(neuron_id))
for neuron_id in range(len(ca1_cont_neuron_ids)):
    all_fullname_yticks.append("CA1_"+str(neuron_id+len(ca1_cue_neuron_ids)))
    all_id_only_yticks.append(str(neuron_id+len(ca1_cue_neuron_ids)))
y_pop_separation.append(len(ca1_cue_neuron_ids) + len(ca1_cont_neuron_ids) + y_pop_separation[len(y_pop_separation)-1])
#   * EC
for neuron_id in range(len(ec_neuron_ids[len(ec_neuron_ids)-1])):
    all_fullname_yticks.append("EC_"+str(neuron_id))
    all_id_only_yticks.append(str(neuron_id))
#y_pop_separation.append(len(ec_neuron_ids[len(ec_neuron_ids)-1]) + y_pop_separation[len(y_pop_separation)-1])

In [ ]:
if experiment >= 0:
    all_ec = False
    print("- Events: " + str(len(events)))
    # Convert events in a more human-readable format
    timestamp_offset = 0
    events_timestamp = [[], [], [], [], [], [], []] # 0 = NOISE, 1 = IN, 2 = DG, 3 = ca3cue, 4 = ca3cont, 5 = ca1, 6 = EC
    events_neuron_id = [[], [], [], [], [], [], []]
    events_neuron_id_formated = [[], [], [], [], [], [], []]
    events_neuron_tag = [[], [], [], [], [], [], []]
    yticks = []
    yticks_pos = []
    labels = ["NOISE", "IN", "DG", "CA3(cue)", "CA3(cont)", "CA1", "EC"]
    sample_num = 0 
    delay = 0.0
    
    # Events recorded during test
    for event in events:
        tag = ""
        if timestamp_offset == 0:
            timestamp_offset = event.timestamp
        # Get timestamp of event in ms from first spike
        timestamp_format_ms = (event.timestamp - timestamp_offset) * 1e-3
        # Round to up if >= x.5
        if timestamp_format_ms - int(timestamp_format_ms) >= 0.5:
            timestamp_format_ms = int(timestamp_format_ms) + 1
        else:
            timestamp_format_ms = int(timestamp_format_ms)
        # Print info
        if len(events) < 100:
            print("  * id=" + str(event.neuron_id) + " t=" + str(timestamp_format_ms) + " ms")
        if event.neuron_id in dg_all_neurons:
            events_timestamp[2].append(timestamp_format_ms)
            events_neuron_id[2].append(event.neuron_id)
            tag = "DG_" + str(dg_all_neurons.index(event.neuron_id))
            events_neuron_tag[2].append(tag)
            event_formated = len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + dg_all_neurons.index(event.neuron_id) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[2].append(event_formated)
        elif event.neuron_id in dg_cont_pop.neuron_ids:
            events_timestamp[2].append(timestamp_format_ms)
            events_neuron_id[2].append(event.neuron_id)
            tag = "DG_" + str(dg_cont_neuron_ids.index(event.neuron_id) + len(dg_all_neurons))
            events_neuron_tag[2].append(tag)
            event_formated = len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + dg_cont_neuron_ids.index(event.neuron_id) + num_dg_neurons + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[2].append(event_formated)
        elif event.neuron_id in ca3cue_pop.neuron_ids:
            events_timestamp[3].append(timestamp_format_ms)
            events_neuron_id[3].append(event.neuron_id)
            tag = "CA3cue_" + str(ca3cue_neuron_ids.index(event.neuron_id))
            events_neuron_tag[3].append(tag)
            event_formated = len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + ca3cue_neuron_ids.index(event.neuron_id) + num_dg_neurons + len(dg_cont_neuron_ids) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[3].append(event_formated)
        elif event.neuron_id in ca3cont_pop.neuron_ids:
            events_timestamp[4].append(timestamp_format_ms)
            events_neuron_id[4].append(event.neuron_id)
            tag = "CA3cont_" + str(ca3cont_neuron_ids.index(event.neuron_id))
            events_neuron_tag[4].append(tag)
            event_formated = len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + ca3cont_neuron_ids.index(event.neuron_id) + num_dg_neurons + len(dg_cont_neuron_ids) + len(ca3cue_neuron_ids) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[4].append(event_formated)
        elif event.neuron_id in ca1_cue_pop.neuron_ids:
            events_timestamp[5].append(timestamp_format_ms)
            events_neuron_id[5].append(event.neuron_id)
            tag = "CA1_" + str(ca1_cue_neuron_ids.index(event.neuron_id))
            events_neuron_tag[5].append(tag)
            event_formated = len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + ca1_cue_neuron_ids.index(event.neuron_id) + num_dg_neurons + len(dg_cont_neuron_ids) + len(ca3cue_neuron_ids) + len(ca3cont_neuron_ids) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[5].append(event_formated)
        elif event.neuron_id in ca1_cont_pop.neuron_ids:
            events_timestamp[5].append(timestamp_format_ms)
            events_neuron_id[5].append(event.neuron_id)
            tag = "CA1_" + str(ca1_cont_neuron_ids.index(event.neuron_id) + len(ca1_cue_neuron_ids))
            events_neuron_tag[5].append(tag)
            event_formated = len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + ca1_cont_neuron_ids.index(event.neuron_id) + num_dg_neurons + len(dg_cont_neuron_ids) + len(ca3cue_neuron_ids) + len(ca3cont_neuron_ids) + len(ca1_cue_neuron_ids) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[5].append(event_formated)
        elif event.neuron_id in in_cue_pop.neuron_ids:
            events_timestamp[1].append(timestamp_format_ms)
            events_neuron_id[1].append(event.neuron_id)
            tag = "IN_" + str(in_cue_neuron_ids.index(event.neuron_id))
            events_neuron_tag[1].append(tag)
            event_formated = in_cue_neuron_ids.index(event.neuron_id) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[1].append(event_formated)
        elif event.neuron_id in in_cont_pop.neuron_ids:
            events_timestamp[1].append(timestamp_format_ms)
            events_neuron_id[1].append(event.neuron_id)
            tag = "IN_" + str(in_cont_neuron_ids.index(event.neuron_id) + len(in_cue_neuron_ids) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids))
            events_neuron_tag[1].append(tag)
            event_formated = len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids) + in_cont_neuron_ids.index(event.neuron_id) + len(in_cue_neuron_ids)
            events_neuron_id_formated[1].append(event_formated)
        elif event.neuron_id in noise_cue_pop.neuron_ids:
            events_timestamp[0].append(timestamp_format_ms)
            events_neuron_id[0].append(event.neuron_id)
            tag = "NOISE_" + str(noise_cue_neuron_ids.index(event.neuron_id))
            events_neuron_tag[0].append(tag)
            event_formated = noise_cue_neuron_ids.index(event.neuron_id)
            events_neuron_id_formated[0].append(event_formated)
        elif event.neuron_id in noise_cont_pop.neuron_ids:
            events_timestamp[0].append(timestamp_format_ms)
            events_neuron_id[0].append(event.neuron_id)
            tag = "NOISE_" + str(noise_cont_neuron_ids.index(event.neuron_id) + len(noise_cue_neuron_ids))
            events_neuron_tag[0].append(tag)
            event_formated = noise_cont_neuron_ids.index(event.neuron_id) + len(noise_cue_neuron_ids)
            events_neuron_id_formated[0].append(event_formated)

        if all_ec and event.neuron_id in ec_all_neuron_ids:
            events_timestamp[6].append(timestamp_format_ms)
            events_neuron_id[6].append(event.neuron_id)
            tag = "EC_" + str(ec_all_neuron_ids.index(event.neuron_id))
            events_neuron_tag[6].append(tag)
            event_formated = len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + ec_all_neuron_ids.index(event.neuron_id) + num_dg_neurons + len(dg_cont_neuron_ids) + len(ca3cue_neuron_ids) + len(ca3cont_neuron_ids) + len(ca1_cue_neuron_ids) + len(ca1_cont_neuron_ids) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[6].append(event_formated)
        if not all_ec and event.neuron_id in ec_pop[len(ec_pop)-1].neuron_ids:
            events_timestamp[6].append(timestamp_format_ms)
            events_neuron_id[6].append(event.neuron_id)
            tag = "EC_" + str(ec_pop[len(ec_pop)-1].neuron_ids.index(event.neuron_id))
            events_neuron_tag[6].append(tag)
            event_formated = len(gen_cue_neuron_ids) + len(gen_cont_neuron_ids) + ec_pop[len(ec_pop)-1].neuron_ids.index(event.neuron_id) + num_dg_neurons + len(dg_cont_neuron_ids) + len(ca3cue_neuron_ids) + len(ca3cont_neuron_ids) + len(ca1_cue_neuron_ids) + len(ca1_cont_neuron_ids) + len(noise_cue_neuron_ids) + len(noise_cont_neuron_ids)
            events_neuron_id_formated[6].append(event_formated)
        if not(tag in yticks) and not(tag == ""):
            yticks.append(tag)
            yticks_pos.append(event_formated)
    # Store events in file 
    #file = open("results/sequential_memory_noise/1/without_noise/events_sequential_noise_"+str(experiment)+"_"+str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))+".txt", "w")
    file = open(path+"events_"+res_exp_name+".txt", "w")
    file.write(str({"timestamp_ms": events_timestamp, "neuron_ids": events_neuron_id, "neuron_ids_formated": events_neuron_id_formated, "event_tag": events_neuron_tag, "y_axis_labels": yticks, "y_axis_labels_pos": yticks_pos, "labels": labels, "all_fullname_y_axis_labels": all_fullname_yticks, "all_id_only_y_axis_labels":all_id_only_yticks, "y_pop_separation": y_pop_separation}))
    file.close()

In [ ]:
# Plot the spikes events in time 
import matplotlib.pyplot as plt

plt.figure(figsize=(20,30))

plt.xlabel("Time (ms)")
plt.ylabel("Neuron ID")
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'black'
max_time = 0
for i in range(len(events_timestamp)):
    if i == 0:
        plt.plot(events_timestamp[i], events_neuron_id_formated[i], "o", label=labels[i], color="tab:pink")
    else:
        plt.plot(events_timestamp[i], events_neuron_id_formated[i], "o", label=labels[i])
        if len(events_timestamp[i]) > 0 and max_time < max(events_timestamp[i]):
            max_time = max(events_timestamp[i])

# Line to separate populations
for y in y_pop_separation:
    plt.hlines(y=y+0.5, xmin=-max_time*0.025, xmax=max_time+max_time*0.025, colors='black', linestyles='-', lw=1)

#plt.yticks(yticks_pos, yticks, weight='bold')
plt.yticks(range(len(all_id_only_yticks)), all_id_only_yticks, weight='bold')

#plt.xlim([-5, 100])
#plt.xlim([4500, 5000])
plt.xlim([-max_time*0.025, max_time+max_time*0.025])
plt.ylim([-0.5, len(all_id_only_yticks)])

plt.legend(bbox_to_anchor =(0.47, 1.018), ncol = len(labels))

#plt.savefig("results/sequential_memory_noise/1/without_noise/sequential_noise_"+str(experiment)+"_"+str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))+".png", bbox_inches='tight')
plt.savefig(path+res_exp_name+".png", bbox_inches='tight')
plt.show()

In [3469]:
# Close the device
close = False
if close:
    samna.device.close_device(model)